In [1]:
import numpy as np
import torch
from snntorch import spikegen
from tqdm import tqdm
from torchvision import datasets, transforms
import matplotlib.pylab as plt
import spynnaker.pyNN as p
transform=transforms.Compose([
            transforms.Resize((28,28)),
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor(),
            transforms.Normalize((0,),(1,))])
dataPath='~/datasets' 
cifar_trainset = datasets.MNIST(dataPath, train=True, download=True, transform = transform)

In [2]:
def convert_to_latency_code(input_data, samples_intervel=5, time_step=5, num_steps=10):# To make SpikeSourceArray 
    input_size  = np.prod(input_data.shape)
    print(input_size)
    slice_interval=time_step
    input_interval=samples_intervel

    number_of_digits=len(input_data)
    coded_digits=[]
    for i in range(number_of_digits):
        s=spikegen.rate((torch.t(input_data[i][0:28,0:28]/255)).float(),num_steps=num_steps)
        coded_digits.append(s[:-1])
    current_time=5#  
    spikes_time_source_array=[[] for _ in range(input_size )]
    with tqdm(total=number_of_digits) as pbar:
        for coded_digit_counter,coded_digit in enumerate(coded_digits):
            for time_slice_counter, time_slice in enumerate(coded_digit):
                timed_flatten_time_slice=time_slice.view(-1)*current_time
                for  spike_time_index in torch.nonzero(timed_flatten_time_slice):
                    spikes_time_source_array[spike_time_index].append(timed_flatten_time_slice[spike_time_index].item())
                current_time+=slice_interval
            current_time+=input_interval
            pbar.update(1)
    return (current_time, spikes_time_source_array)

In [3]:
input_data = cifar_trainset.data[1]
print(f'input_data shape: {input_data.shape}')
plt.figure(figsize=(1,1))
plt.imshow(input_data)

input_data shape: torch.Size([28, 28])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## To show an application, I added some s&p noise to data to see how a average filter can reduce the amount of noise using the conve layer

In [4]:
#adding noise in data
noise_factor = 0.07

from skimage.util import random_noise
noisy_input_data = random_noise(input_data, mode="s&p",clip=True, amount=noise_factor)
plt.figure(figsize=(1,1))
plt.imshow(noisy_input_data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
run_time, SSA = convert_to_latency_code(torch.tensor(noisy_input_data).unsqueeze(0), time_step=5, num_steps= 1000 )

784


100%|██████████| 1/1 [00:00<00:00, 35.71it/s]


In [6]:
################################################# Custum convolution parameters (kernel and stride/ no padding is implemented) 
kernel = np.array([[ 1, 1, 1],
             [ 1, 1, 1],
             [ 1, 1, 1]])#*1/16

kernel*=10
stride = 1

################################################# Connection List

input_size = input_data.shape
input_as_index_array = np.arange(np.prod(input_size))
input_as_index_array = input_as_index_array.reshape(input_size)


conv_connection_list = []
post = 0
out_size = 0
for y in range(0, input_as_index_array.shape[1]-kernel.shape[1]+1, stride):
    for x in range(0, input_as_index_array.shape[0]-kernel.shape[0]+1, stride):
        sub_mat = input_as_index_array[y:y+kernel.shape[0], x:x+kernel.shape[0]]
        out_size +=1
        for i, pre in enumerate(sub_mat.flatten()):
            tup = ( pre, post, kernel.flatten()[i], .1)
            conv_connection_list.append(tup)
        post +=1
print(f'output size: {int(np.sqrt(out_size))}*{int(np.sqrt(out_size))}')
output_size_x, output_size_y = int(np.sqrt(out_size)), int(np.sqrt(out_size))

output size: 26*26


In [7]:
excitatory_neuron_parameters = {'tau_m': 20.0,
                                 'cm': 1.0,
                                 'v_rest': -65.0,
                                 'v_reset': -65.0,
                                 'v_thresh': -63.0,
                                 'tau_syn_E': 5.0,
                                 'tau_syn_I': 5.0,
                                 'tau_refrac': 0.1,
                                 'i_offset': 0.0}

In [ ]:
p.setup(timestep = 1)
ssa = p.Population(np.prod(input_size), p.SpikeSourceArray(spike_times=SSA), label = 'pop_ssa')


pop_out = p.Population(output_size_x*output_size_y,
                      p.IF_curr_exp(**excitatory_neuron_parameters),
                      label = 'pop_xy')
ssa_out_prj = p.Projection(ssa, 
             pop_out, 
             p.FromListConnector(conv_connection_list),
             p.StaticSynapse())

ssa.record('spikes')
pop_out.record('spikes')
p.run(run_time)


In [10]:
output_spike_trains=[pop_out.get_data("spikes").segments[0].spiketrains[i][:] for i in range(output_size_x*output_size_y)]


In [11]:
output = np.array([len(s) for s in output_spike_trains])
plt.figure(figsize=(1,1))
plt.imshow(noisy_input_data)
plt.figure(figsize=(1,1))
plt.imshow(np.array(output).reshape((output_size_x,output_size_y)).T)#int(np.sqrt(out_size)), int(np.sqrt(out_size))))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …